In [ ]:
# PRE-STEP: Install Required Dependencies
%pip install langchain
%pip install langgraph
%pip install langchain-openai
%pip install chromadb
%pip install python-dotenv
%pip install pydantic

In [ ]:
# PRE-STEP: Import Core Libraries and Configure Environment
import os
from datetime import datetime
from typing import List, Dict, TypedDict, Annotated, Sequence
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import BaseMessage
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

# Initialize environment and models
load_dotenv(dotenv_path='env.txt')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings()
output_parser = StrOutputParser()

# State and memory structures
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    working_memory: dict
    episodic_recall: list
    semantic_facts: dict
    user_id: str
    conversation_id: str

class SemanticFact(BaseModel):
    subject: str = Field(description="Entity or topic")
    predicate: str = Field(description="Relationship or property")
    object: str = Field(description="Value or related entity")
    confidence: float = Field(description="Confidence score 0-1")
    source: str = Field(description="Source: user or assistant")

# Initialize vector store
vector_store = Chroma(
    collection_name="agent_memory",
    embedding_function=embeddings,
    persist_directory="./memory_store"
)

# Episodic memory functions
def store_episodic_memory(vector_store, conversation_id: str, messages: List, summary: str = None):
    if not summary and messages:
        summary = f"Conversation about: {messages[0][1] if isinstance(messages[0], tuple) else messages[0].content[:100]}..."
    metadata = {
        "type": "episodic",
        "conversation_id": conversation_id,
        "timestamp": datetime.now().isoformat(),
        "message_count": len(messages)
    }
    conversation_text = ""
    for msg in messages:
        conversation_text += f"{msg[0]}: {msg[1]}\n" if isinstance(msg, tuple) else f"{msg.type}: {msg.content}\n"
    vector_store.add_documents([Document(page_content=conversation_text, metadata=metadata)])
    return conversation_id

def retrieve_episodic_memories(vector_store, query: str, k: int = 3):
    return vector_store.similarity_search(query=query, k=k, filter={"type": {"$eq": "episodic"}})

# Semantic memory functions
def extract_semantic_facts(messages: List) -> List[SemanticFact]:
    extraction_prompt = PromptTemplate.from_template("""
    Analyze this conversation and extract important factual statements.
    Conversation: {conversation}
    Extract facts in JSON format:
    {{"facts": [{{"subject": "...", "predicate": "...", "object": "...", 
                  "confidence": 0.0-1.0, "source": "user or assistant"}}]}}
    Only extract clear facts. Output valid JSON only.
    """)
    conversation_text = ""
    for msg in messages:
        conversation_text += f"{msg[0]}: {msg[1]}\n" if isinstance(msg, tuple) else f"{msg.type}: {msg.content}\n"
    try:
        result = (extraction_prompt | llm | JsonOutputParser()).invoke({"conversation": conversation_text})
        return [SemanticFact(**fact_dict) for fact_dict in result.get("facts", [])]
    except Exception as e:
        print(f"Fact extraction error: {e}")
        return []

def store_semantic_facts(vector_store, facts: List[SemanticFact], user_id: str = "default"):
    documents = []
    for fact in facts:
        documents.append(Document(
            page_content=f"{fact.subject} {fact.predicate} {fact.object}",
            metadata={
                "type": "semantic", "user_id": user_id,
                "subject": fact.subject, "predicate": fact.predicate,
                "object": fact.object, "confidence": fact.confidence,
                "timestamp": datetime.now().isoformat()
            }
        ))
    if documents:
        vector_store.add_documents(documents)
    return len(documents)

def retrieve_semantic_facts(vector_store, query: str, user_id: str = "default", k: int = 5):
    results = vector_store.similarity_search(
        query=query, k=k,
        filter={"$and": [{"type": {"$eq": "semantic"}}, {"user_id": {"$eq": user_id}}]}
    )
    return [{
        "subject": doc.metadata.get("subject"),
        "predicate": doc.metadata.get("predicate"),
        "object": doc.metadata.get("object"),
        "confidence": doc.metadata.get("confidence", 1.0)
    } for doc in results]

def format_semantic_context(facts: List[Dict]) -> str:
    if not facts:
        return "No relevant facts found."
    context = "Known facts:\n"
    for fact in facts:
        if fact.get('confidence', 1.0) > 0.7:
            context += f"- {fact['subject']} {fact['predicate']} {fact['object']}\n"
    return context

# Unified memory agent
def unified_memory_agent(state: AgentState) -> dict:
    current_messages = state.get("messages", [])
    user_id = state.get("user_id", "default")
    conversation_id = state.get("conversation_id", f"conv_{datetime.now().timestamp()}")
    
    episodic_context = ""
    semantic_context = ""
    
    if current_messages:
        latest_query = current_messages[-1][1] if isinstance(current_messages[-1], tuple) else current_messages[-1].content
        
        # Retrieve memories
        past_episodes = retrieve_episodic_memories(vector_store, latest_query, k=2)
        if past_episodes:
            episodic_context = "Relevant past conversations:\n"
            for episode in past_episodes:
                timestamp = episode.metadata.get('timestamp', 'Unknown')
                episodic_context += f"[{timestamp}]:\n{episode.page_content[:200]}...\n\n"
        
        facts = retrieve_semantic_facts(vector_store, latest_query, user_id=user_id, k=3)
        semantic_context = format_semantic_context(facts)
    
    # Generate response with memory context
    memory_prompt = PromptTemplate.from_template("""
You are an AI assistant with both episodic and semantic memory.
{semantic_context}
{episodic_context}
Current conversation:
{messages}
Respond using your memories when relevant. Be consistent with known facts and past conversations.
""")
    
    formatted_messages = ""
    for msg in current_messages[-5:] if current_messages else []:
        formatted_messages += f"{msg[0]}: {msg[1]}\n" if isinstance(msg, tuple) else f"{msg.type}: {msg.content}\n"
    
    response = (memory_prompt | llm | output_parser).invoke({
        "semantic_context": semantic_context,
        "episodic_context": episodic_context,
        "messages": formatted_messages
    })
    
    # Store memories
    if len(current_messages) >= 2:
        store_episodic_memory(vector_store, conversation_id, current_messages)
    
    if current_messages:
        messages_with_response = current_messages + [("assistant", response)]
        new_facts = extract_semantic_facts(messages_with_response[-3:])
        if new_facts:
            stored = store_semantic_facts(vector_store, new_facts, user_id)
            state["semantic_facts"] = {"extracted": stored}
    
    return {
        "messages": [("assistant", response)],
        "episodic_recall": past_episodes if past_episodes else [],
        "semantic_facts": state.get("semantic_facts", {})
    }

# Build and compile workflow
memory_workflow = StateGraph(AgentState)
memory_workflow.add_node("memory_agent", unified_memory_agent)
memory_workflow.set_entry_point("memory_agent")
memory_workflow.add_edge("memory_agent", END)
memory_app = memory_workflow.compile()

In [5]:
# PRE-STEP: Establish facts
test_1 = {
    "messages": [("user", "Hi, I'm Sarah Chen. I'm a data scientist working on climate models. I'm vegetarian and prefer concise technical explanations.")],
    "user_id": "sarah_chen",
    "conversation_id": "conv_001"
}

result_1 = memory_app.invoke(test_1)
print("Response 1:")
print(result_1["messages"][-1][1] if isinstance(result_1["messages"][-1], tuple) else result_1["messages"][-1].content)

print("\n" + "="*50 + "\n")

# Test 2: Use semantic memory
test_2 = {
    "messages": [("user", "What machine learning techniques would you recommend for time series forecasting?")],
    "user_id": "sarah_chen",
    "conversation_id": "conv_002"
}

result_2 = memory_app.invoke(test_2)
print("Response 2 (with semantic memory):")
print(result_2["messages"][-1][1] if isinstance(result_2["messages"][-1], tuple) else result_2["messages"][-1].content)

print("\n" + "="*50 + "\n")

# Test 3: Reference past conversation and facts
test_3 = {
    "messages": [("user", "Can you recommend some lunch options for our team meeting?")],
    "user_id": "sarah_chen",
    "conversation_id": "conv_003"
}

result_3 = memory_app.invoke(test_3)
print("Response 3 (with full memory context):")
print(result_3["messages"][-1][1] if isinstance(result_3["messages"][-1], tuple) else result_3["messages"][-1].content)

Response 1:
Hi Sarah! It's great to meet you. As a data scientist working on climate models, you must have a fascinating job. If you have any specific questions or topics you'd like to discuss, feel free to ask!


Response 2 (with semantic memory):
For time series forecasting, the choice of machine learning techniques can depend on the nature of your data and the patterns you want to capture. Here are some recommendations based on known facts:

1. **XGBoost/LightGBM**: These gradient boosting methods are effective, especially when you engineer time-based features such as lags and rolling statistics. They can handle non-linear relationships well and are often used in competitions for their performance.

2. **ARIMA/SARIMA**: If your data exhibits linear patterns and seasonality, traditional statistical models like ARIMA or SARIMA can be very effective. They are particularly useful for datasets where the underlying relationships are more linear.

3. **LSTM (Long Short-Term Memory) or TCN 

In [13]:
# Step 1: Install LangMem and Additional Dependencies
%pip install -q langgraph-checkpoint


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Step 2: Import Components and Create Memory Infrastructure
from langgraph.checkpoint.memory import MemorySaver
import hashlib
from collections import deque

In [ ]:
# Step 3: Create Custom Memory Management Tools
class SimpleMemoryStore:
    """Lightweight memory store for demonstration"""
    
    def __init__(self):
        self.memories = {}
        self.memory_index = []
        
    def add_memory(self, content: str, user_id: str = "default", metadata: dict = None):
        """Add a memory to the store"""
        memory_id = hashlib.md5(f"{content}{datetime.now().isoformat()}".encode()).hexdigest()[:8]
        memory_entry = {
            "id": memory_id,
            "content": content,
            "user_id": user_id,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {}
        }
        self.memories[memory_id] = memory_entry
        self.memory_index.append((memory_id, content))
        return memory_id
    
    def search_memories(self, query: str, user_id: str = "default", k: int = 3):
        """Simple keyword-based memory search"""
        results = []
        query_lower = query.lower()
        
        for memory_id, content in self.memory_index:
            memory = self.memories.get(memory_id)
            if memory and memory["user_id"] == user_id:
                # Simple relevance scoring based on keyword overlap
                score = sum(1 for word in query_lower.split() if word in content.lower())
                if score > 0:
                    results.append((score, memory))
        
        # Sort by relevance and return top k
        results.sort(key=lambda x: x[0], reverse=True)
        return [memory for _, memory in results[:k]]

# Create memory store instance
memory_store = SimpleMemoryStore()

# Create tool functions that mimic LangMem's interface
def manage_memory_tool(memories: list, user_id: str = "default"):
    """Tool to store memories"""
    stored_ids = []
    for memory in memories:
        if isinstance(memory, dict):
            content = memory.get("content", str(memory))
            metadata = memory.get("metadata", {})
        else:
            content = str(memory)
            metadata = {}
        
        memory_id = memory_store.add_memory(content, user_id, metadata)
        stored_ids.append(memory_id)
    
    return {"stored": len(stored_ids), "ids": stored_ids}

def search_memory_tool(query: str, user_id: str = "default"):
    """Tool to search memories"""
    results = memory_store.search_memories(query, user_id)
    return results

In [ ]:
# Step 4: Create Procedural Memory Manager
class ProceduralMemoryManager:
    """Manages procedural memory and prompt optimization"""
    
    def __init__(self):
        self.performance_log = deque(maxlen=100)  # Keep last 100 interactions
        self.current_rules = []
        self.rule_performance = {}  # Track rule effectiveness
        
    def log_interaction(self, query: str, response: str, success: bool, feedback: str = None):
        """Log interaction for procedural learning"""
        entry = {
            "timestamp": datetime.now().isoformat(),
            "query": query,
            "response": response[:200],
            "success": success,
            "feedback": feedback,
            "active_rules": [r["condition"] for r in self.current_rules[:3]]
        }
        self.performance_log.append(entry)
        
        # Update rule performance metrics
        for rule_condition in entry["active_rules"]:
            if rule_condition not in self.rule_performance:
                self.rule_performance[rule_condition] = {"success": 0, "total": 0}
            self.rule_performance[rule_condition]["total"] += 1
            if success:
                self.rule_performance[rule_condition]["success"] += 1
        
        # Trigger rule extraction after every 5 interactions
        if len(self.performance_log) % 5 == 0:
            self.extract_procedural_rules()
    
    def extract_procedural_rules(self):
        """Analyze logs to extract behavioral improvements"""
        if len(self.performance_log) < 3:
            return
        
        recent_logs = list(self.performance_log)[-10:]
        successful = [log for log in recent_logs if log["success"]]
        failed = [log for log in recent_logs if not log["success"]]
        
        extraction_prompt = PromptTemplate.from_template("""
        Analyze these interaction logs and extract procedural rules to improve behavior.
        
        Successful interactions ({success_count}):
        {successful_examples}
        
        Failed interactions ({fail_count}):
        {failed_examples}
        
        Extract 1-3 specific behavioral rules in this format:
        {{"rules": [
            {{"condition": "If user asks for explanation", 
              "action": "Then provide structured response with examples", 
              "priority": 1-10}}
        ]}}
        
        Focus on clear, actionable patterns.
        Output valid JSON only.
        """)
        
        try:
            result = (extraction_prompt | llm | JsonOutputParser()).invoke({
                "success_count": len(successful),
                "successful_examples": "\n".join([f"- Query: {s['query']}" for s in successful[:3]]),
                "fail_count": len(failed),
                "failed_examples": "\n".join([f"- Query: {f['query']}" for f in failed[:3]])
            })
            
            new_rules = result.get("rules", [])
            if new_rules:
                # Add performance tracking for new rules
                for rule in new_rules:
                    rule["performance"] = {"success": 0, "total": 0}
                
                self.current_rules.extend(new_rules)
                # Keep only highest priority, best performing rules
                self.current_rules = sorted(
                    self.current_rules, 
                    key=lambda x: (x.get("priority", 0), self._get_rule_success_rate(x)),
                    reverse=True
                )[:5]
                
                print(f"✓ Extracted {len(new_rules)} new procedural rules")
        except Exception as e:
            print(f"Rule extraction error: {e}")
    
    def _get_rule_success_rate(self, rule):
        """Calculate success rate for a rule"""
        perf = self.rule_performance.get(rule["condition"], {"success": 0, "total": 1})
        return perf["success"] / max(perf["total"], 1)
    
    def get_active_rules(self) -> str:
        """Format current rules for prompt injection"""
        if not self.current_rules:
            return ""
        
        rules_text = "PROCEDURAL RULES (learned from experience):\n"
        for i, rule in enumerate(self.current_rules[:3], 1):  # Use top 3 rules
            success_rate = self._get_rule_success_rate(rule)
            rules_text += f"{i}. {rule['condition']} → {rule['action']} (success: {success_rate:.0%})\n"
        return rules_text

# Initialize procedural manager
procedural_manager = ProceduralMemoryManager()


In [17]:
# SStep 5: Create Memory-Enhanced Agent with Procedural Learning
def procedural_agent(state: AgentState) -> dict:
    """Agent with procedural memory and hot-path memory tools"""
    
    current_messages = state.get("messages", [])
    user_id = state.get("user_id", "default")
    
    if not current_messages:
        return {"messages": [("assistant", "Hello! How can I help you today?")]}
    
    # Get latest query
    latest_query = current_messages[-1][1] if isinstance(current_messages[-1], tuple) else current_messages[-1].content
    
    # Search relevant memories (hot-path)
    memory_results = search_memory_tool(latest_query, user_id)
    memory_context = ""
    if memory_results:
        memory_context = "Relevant memories:\n"
        for mem in memory_results[:2]:
            memory_context += f"- {mem['content'][:100]}...\n"
    
    # Get active procedural rules
    procedural_rules = procedural_manager.get_active_rules()
    
    # Create enhanced prompt with procedural memory
    enhanced_prompt = PromptTemplate.from_template("""
You are an adaptive AI assistant that learns from experience.

{procedural_rules}

{memory_context}

Current conversation:
{messages}

Important: Follow the procedural rules above to optimize your response.
Response:""")
    
    # Format messages
    formatted_messages = ""
    for msg in current_messages[-5:] if current_messages else []:
        formatted_messages += f"{msg[0]}: {msg[1]}\n" if isinstance(msg, tuple) else f"{msg.type}: {msg.content}\n"
    
    # Generate response
    response = (enhanced_prompt | llm | output_parser).invoke({
        "procedural_rules": procedural_rules,
        "memory_context": memory_context,
        "messages": formatted_messages
    })
    
    # Store interaction in memory (hot-path)
    memory_entry = {
        "content": f"Q: {latest_query}\nA: {response[:200]}",
        "metadata": {"type": "interaction", "timestamp": datetime.now().isoformat()}
    }
    manage_memory_tool([memory_entry], user_id)
    
    # Log for procedural learning
    # Simulate success based on response quality indicators
    success = (
        len(response) > 50 and 
        len(response) < 500 and 
        not response.endswith("?") and
        ("example" in response.lower() or "step" in response.lower() or len(response.split('\n')) > 2)
    )
    
    procedural_manager.log_interaction(latest_query, response, success)
    
    return {
        "messages": [("assistant", response)],
        "working_memory": {"memory_stored": True, "success": success}
    }

# Create workflow
procedural_workflow = StateGraph(AgentState)
procedural_workflow.add_node("procedural_agent", procedural_agent)
procedural_workflow.set_entry_point("procedural_agent")
procedural_workflow.add_edge("procedural_agent", END)

procedural_app = procedural_workflow.compile()


In [18]:
# Step 6: Background Memory Optimization
def optimize_memories_background():
    """Background task to optimize memories and extract patterns"""
    
    print("\n🔄 Running background memory optimization...")
    
    # Extract and refine procedural rules
    procedural_manager.extract_procedural_rules()
    
    # Display performance metrics
    if procedural_manager.current_rules:
        print(f"📊 Active procedural rules: {len(procedural_manager.current_rules)}")
        for i, rule in enumerate(procedural_manager.current_rules[:3], 1):
            success_rate = procedural_manager._get_rule_success_rate(rule)
            print(f"   {i}. {rule['condition'][:50]}...")
            print(f"      → {rule['action'][:50]}...")
            print(f"      Performance: {success_rate:.0%}")
    
    # Memory consolidation (simulate)
    print(f"💾 Total memories stored: {len(memory_store.memories)}")
    
    return {"status": "optimization_complete"}



In [19]:
# Step 7: Test Procedural Memory System
print("🚀 Testing Procedural Memory System\n")

# Test 1: Initial interaction (no rules yet)
test_1 = {
    "messages": [("user", "Explain quantum computing")],
    "user_id": "demo_user"
}

result_1 = procedural_app.invoke(test_1)
print("Response 1 (no procedural rules):")
print(result_1["messages"][-1][1][:200] if isinstance(result_1["messages"][-1], tuple) else result_1["messages"][-1].content[:200])
print("...")

print("\n" + "="*60 + "\n")

# Simulate training interactions
print("📚 Training phase - simulating multiple interactions...\n")

training_data = [
    ("What's machine learning?", True),  # Good explanation expected
    ("Hello", False),  # Too simple
    ("Explain neural networks", True),  # Good explanation expected
    ("What time is it?", False),  # Can't answer
    ("How does encryption work?", True),  # Good explanation expected
    ("Thanks", False),  # Too simple
]

for query, expected_success in training_data:
    test = {"messages": [("user", query)], "user_id": "demo_user"}
    result = procedural_app.invoke(test)
    print(f"• Query: {query[:30]}... Success: {result['working_memory']['success']}")

# Run background optimization
optimize_memories_background()

print("\n" + "="*60 + "\n")

# Test 2: After learning
test_2 = {
    "messages": [("user", "Tell me about blockchain technology")],
    "user_id": "demo_user"
}

result_2 = procedural_app.invoke(test_2)
print("Response 2 (with procedural rules):")
print(result_2["messages"][-1][1] if isinstance(result_2["messages"][-1], tuple) else result_2["messages"][-1].content)

print("\n" + "="*60 + "\n")

# Test 3: Memory recall
test_3 = {
    "messages": [("user", "What technical topics have we discussed?")],
    "user_id": "demo_user"
}

result_3 = procedural_app.invoke(test_3)
print("Response 3 (memory recall):")
print(result_3["messages"][-1][1] if isinstance(result_3["messages"][-1], tuple) else result_3["messages"][-1].content)

🚀 Testing Procedural Memory System

Response 1 (no procedural rules):
Quantum computing is a type of computation that leverages the principles of quantum mechanics to process information in fundamentally different ways than classical computers. Here are some key concept
...


📚 Training phase - simulating multiple interactions...

• Query: What's machine learning?... Success: False
• Query: Hello... Success: False
• Query: Explain neural networks... Success: False
✓ Extracted 3 new procedural rules
• Query: What time is it?... Success: False
• Query: How does encryption work?... Success: False
• Query: Thanks... Success: False

🔄 Running background memory optimization...
✓ Extracted 3 new procedural rules
📊 Active procedural rules: 5
   1. If user asks about a technical topic like machine ...
      → Then break down the concept into simple terms and ...
      Performance: 0%
   2. If user asks about a technical topic...
      → Then break down the topic into simpler terms and p...
    